In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import scipy.stats as stats

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, scale
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [ ]:
#Unpickle file 
modeling_df = pd.read_pickle("./cleaned_data.pkl")

In [ ]:
df = modeling_df

## Feature Engineering

In [ ]:
X = df.iloc[:,2:] #'walk_score', 'res_score', 'attraction_score','num_reviews', 'num_QA', 'num_Tips', 'num_rooms', 'min_price','max_price', 'avg_price

y = df['rating']

### train/test/split

In [ ]:
#first split 20%
X_train_val, X_test, y_train_val, y_test = \
train_test_split(X,y,test_size=0.2, random_state = 42)

In [ ]:
#second split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.25, random_state=3)

In [ ]:
#set up the 3 models to compare:

lm = LinearRegression()

#Feature scaling for train, val, and test so that we can run our ridge model on each
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_val_scaled = scaler.transform(X_val.values)
X_test_scaled = scaler.transform(X_test.values)

lm_reg = Ridge(alpha=1)

#Feature transforms for train, val, and test so that we can run our poly model on each
poly = PolynomialFeatures(degree=2) 

X_train_poly = poly.fit_transform(X_train.values)
X_val_poly = poly.transform(X_val.values)
X_test_poly = poly.transform(X_test.values)

lm_poly = LinearRegression()

In [ ]:
#validate #3 models

lm.fit(X_train, y_train)
print(f'Linear Regression val R^2: {lm.score(X_val, y_val):.3f}')

lm_reg.fit(X_train_scaled, y_train)
print(f'Ridge Regression val R^2: {lm_reg.score(X_val_scaled, y_val):.3f}')

lm_poly.fit(X_train_poly, y_train)
print(f'Degree 2 polynomial regression val R^2: {lm_poly.score(X_val_poly, y_val):.3f}')

In [ ]:
#cross validation
from sklearn.model_selection import KFold

X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=10) #hold out 20% of the data for final testing

#this helps with the way kf will generate indices below
X, y = np.array(X), np.array(y)

In [ ]:
#run the CV

kf = KFold(n_splits=5, shuffle=True, random_state = 71)
cv_lm_r2s, cv_lm_reg_r2s = [], [] #collect the validation results for both models

for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    #simple linear regression
    lm = LinearRegression()
    lm_reg = Ridge(alpha=1)

    lm.fit(X_train, y_train)
    cv_lm_r2s.append(lm.score(X_val, y_val))
    
    #ridge with feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    lm_reg.fit(X_train_scaled, y_train)
    cv_lm_reg_r2s.append(lm_reg.score(X_val_scaled, y_val))

print('Simple regression scores: ', cv_lm_r2s)
print('Ridge scores: ', cv_lm_reg_r2s, '\n')

print(f'Simple mean cv r^2: {np.mean(cv_lm_r2s):.3f} +- {np.std(cv_lm_r2s):.3f}')
print(f'Ridge mean cv r^2: {np.mean(cv_lm_reg_r2s):.3f} +- {np.std(cv_lm_reg_r2s):.3f}')

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

lm_reg = Ridge(alpha=1)
lm_reg.fit(X_scaled,y)
print(f'Ridge Regression test R^2: {lm_reg.score(X_test_scaled, y_test):.3f}')

### Ridge Regression

In [ ]:
y = df.rating
X_= df.drop(['rating','walk_perfect', 'hotel_name'], axis = 1)
X = pd.concat([X_, dummies[['walk_perfect']]], axis = 1)
X.info()

In [ ]:
alphas = 10**np.linspace(10,-2,100)*0.5

In [ ]:
ridge = Ridge(normalize = True)
coefs = []

for a in alphas:
    ridge.set_params(alpha = a)
    ridge.fit(X, y)
    coefs.append(ridge.coef_)
    
np.shape(coefs)

In [ ]:
ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')

In [ ]:
#Split the data 60 - 20 - 20 train/val/test

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=43)

In [ ]:
ridge2 = Ridge(alpha = 4, normalize = True)
ridge2.fit(X_train, y_train)             # Fit a ridge regression on the training data
pred2 = ridge2.predict(X_test)           # Use this model to predict the test data
print(pd.Series(ridge2.coef_, index = X.columns)) # Print coefficients
print(mean_squared_error(y_test, pred2))

In [ ]:
#using CV to choose alpha
ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(X_train, y_train)
ridgecv.alpha_

In [ ]:
ridge4 = Ridge(alpha = ridgecv.alpha_, normalize = True)
ridge4.fit(X_train, y_train)
mean_squared_error(y_test, ridge4.predict(X_test))

In [ ]:
ridge4.fit(X, y)
pd.Series(ridge4.coef_, index = X.columns)

### OLS model

In [ ]:
#-walk_score
y, X = patsy.dmatrices('rating ~  walk_perfect + attraction_score+num_QA+num_rooms+min_price+ max_price+avg_price', data=df, return_type="dataframe")
#hotel_name', 'rating', 'walk_score', 'res_score', 'attraction_score','num_reviews', 'num_QA', 'num_Tips', 'num_rooms', 'min_price','max_price', 'avg_price'

model = sm.OLS(y, X)

# Fit your model to your training set
fit = model.fit()

# Print summary statistics of the model's performance
fit.summary()

### OLS logged

In [ ]:
#using logged variables - walk_score - res_score - log_num_reviews - num_QA - attraction_score - walk_perfect

y, X = patsy.dmatrices('rating ~  log_num_rooms + log_min_price + log_num_Tips + log_avg_price + max_price + min_price', data=df, return_type="dataframe")
#hotel_name', 'rating', 'walk_score', 'res_score', 'attraction_score','num_reviews', 'num_QA', 'num_Tips', 'num_rooms', 'min_price','max_price', 'avg_price'

model = sm.OLS(y, X)

# Fit your model to your training set
fit_log = model.fit()

# Print summary statistics of the model's performance
fit_log.summary()

### Assumptions

In [ ]:
def diagnostic_plot(x, y):
    plt.figure(figsize=(20,5))
   
    rgr = LinearRegression()
    rgr.fit(x,y)
    pred = rgr.predict(x)

    plt.subplot(1, 3, 1)
    plt.scatter(x,y)
    plt.plot(x, fit_log.predict(), color='blue',linewidth=1)
    plt.title("Regression fit")
    plt.xlabel("x")
    plt.ylabel("y")
    
    plt.subplot(1, 3, 2)
    res = y - pred
    plt.scatter(fit_log.predict(), fit_log.resid)
    plt.title("Residual plot")
    plt.xlabel("prediction")
    plt.ylabel("residuals")
    
    plt.subplot(1, 3, 3)
   
    stats.probplot(fit_log, dist="norm", plot=plt)
    plt.title("Normal Q-Q plot")
    
#X = df.iloc[:, 2:]
#y = df.iloc[:, 1]
diagnostic_plot(x, y)

In [ ]:
stats.probplot(df['resid'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot")
plt.show()

In [ ]:
X = df.iloc[:, 2:]
y = df.iloc[:, 1]

lr = LinearRegression()
lr.fit(X,y)

df['predict']=lr.predict(X)
df['resid']=y - df.predict
with sns.axes_style('white'):
    plot=df.plot(kind='scatter',
                  x='predict',y='resid',alpha=0.2,figsize=(10,6))

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(fit_log.predict(), fit_log.resid);

### sklearn

In [ ]:
lr = LinearRegression()

# Choose the predictor variables, here all but the first which is the response variable
# This model is analogous to the Y ~ X1 + X2 + X3 + X4 + X5 + X6 model
X = df.iloc[:, 2:]

# Choose the response variable(s)
y = df.iloc[:, 1]

# Fit the model to the full dataset
lr.fit(X, y)

# Print out the R^2 for the model against the full dataset
lr.score(X,y)